In [19]:
import time
notebook_start = time.time()

In [20]:
# Cell 1: Complete Environment Setup
# ========================================================

# 1. Initial imports and setup
import sys
import os
import shutil
import warnings
warnings.filterwarnings('ignore')

# 2. Clean up existing installations
print("=== Cleaning up existing installations ===")
!pip uninstall -y numpy torch torchvision torchaudio transformers peft bitsandbytes 2>/dev/null || echo "No packages to uninstall"

# Remove problematic directories
problematic_path = "/usr/local/lib/python3.11/dist-packages/~vidia-cudnn-cu12"
if os.path.exists(problematic_path):
    shutil.rmtree(problematic_path)

# Clear pip cache
!pip cache purge

=== Cleaning up existing installations ===
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: torch 2.2.1+cu121
Uninstalling torch-2.2.1+cu121:
  Successfully uninstalled torch-2.2.1+cu121
Found existing installation: torchvision 0.17.1+cu121
Uninstalling torchvision-0.17.1+cu121:
  Successfully uninstalled torchvision-0.17.1+cu121
Found existing installation: torchaudio 2.2.1+cu121
Uninstalling torchaudio-2.2.1+cu121:
  Successfully uninstalled torchaudio-2.2.1+cu121
Files removed: 70 (790.5 MB)


In [21]:
# 3. Install core packages with precise version control
print("\n=== Installing core packages ===")
!pip install -q --upgrade pip
!pip install -q --force-reinstall numpy==1.26.4

# Force reload numpy
if 'numpy' in sys.modules:
    del sys.modules['numpy']
import numpy as np
print(f"NumPy version: {np.__version__}")

# 4. Install PyTorch with CUDA 12.1
print("\n=== Installing PyTorch ===")
!pip install -q torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu121

# 5. Install transformer ecosystem with compatible versions
print("\n=== Installing transformer packages ===")
!pip install -q \
    transformers==4.41.2 \
    peft==0.10.0 \
    datasets==2.18.0 \
    accelerate==0.29.1 \
    einops==0.7.0 \
    scipy==1.13.0 \
    scikit-learn==1.3.2 \
    matplotlib==3.8.0 \
    pandas==2.2.2

# 6. Install bitsandbytes with GPU support
print("\n=== Installing bitsandbytes ===")
try:
    !pip install -q bitsandbytes==0.43.0
except:
    print("Failed to install bitsandbytes with GPU support, installing CPU-only version")
    !pip install -q bitsandbytes-cpu==0.43.0

# 7. Verify installations
print("\n=== Verifying installations ===")
import torch
import torchvision

print("\n=== Core Package Versions ===")
print(f"Python: {sys.version}")
print(f"NumPy: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"Torchvision: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"\n=== CUDA Information ===")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")


=== Installing core packages ===
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 91.7 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.29.1 requires torch>=1.10.0, which is not installed.
easyocr 1.7.2 requires torch, which is not installed.
easyocr 1.7.2 requires torchvision>=0.5, which is not installed.
torchmetrics 1.7.1 requires torch>=2.0.0, which is not installed.
pytorch-lightning 2.5.1.post0 requires torch>=2.1.0, which is not installed.
kaggle-environments 1.16.11 requires transformers>=4.33.1, which is not installed.
stable-baselines3 2.1.0 requires torch>=1.13, which is not installed.
sentence-transformers 3.4.1 requires torch>=1.11.0, which is not installed.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, which is not installed.
fastai 2.7.19 requires torch<2.7,>=1.10, which is not insta

In [22]:
# 8. Import transformer packages
try:
    from datasets import Dataset, load_dataset
    from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
    from transformers import (
        AutoModelForCausalLM,
        AutoTokenizer,
        TrainingArguments,
        Trainer,
        BitsAndBytesConfig,
        pipeline
    )
    print("\n=== Transformer Packages Loaded Successfully ===")
except ImportError as e:
    print(f"\nError importing transformer packages: {e}")
    raise

# 9. Additional verification
try:
    import scipy
    import sklearn
    print(f"\nAdditional Package Versions:")
    print(f"SciPy: {scipy.__version__}")
    print(f"scikit-learn: {sklearn.__version__}")
except ImportError as e:
    print(f"\nWarning: Could not verify some package versions - {e}")

# 10. Clean up warnings
warnings.filterwarnings('ignore', message='.*The installed version of bitsandbytes.*')
warnings.filterwarnings('ignore', message='.*was compiled without GPU support.*')


=== Transformer Packages Loaded Successfully ===

Additional Package Versions:
SciPy: 1.15.2
scikit-learn: 1.2.2


In [23]:
# Cell 2: Model Loading with Robust Error Handling
# ================================================

MODEL_NAME = "gpt2"  # Default model

def print_memory():
    """Memory usage diagnostics"""
    import psutil
    if torch.cuda.is_available():
        gpu_mem = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU Memory: {gpu_mem:.2f}GB", end=" | ")
    ram = psutil.virtual_memory()
    print(f"RAM: {ram.percent}% ({ram.used/1024**3:.1f}/{ram.total/1024**3:.1f}GB)")

def load_model(model_name):
    """Improved model loading with fallback options"""
    print(f"\n=== Loading Model: {model_name} ===")
    print_memory()
    
    trust_remote_code = True
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    
    # Try loading with different configurations
    configs_to_try = [
        {
            "name": "4-bit quantized",
            "quantization_config": BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch_dtype
            ),
            "device_map": "auto"
        },
        {
            "name": "8-bit quantized",
            "quantization_config": BitsAndBytesConfig(load_in_8bit=True),
            "device_map": "auto"
        },
        {
            "name": "Full precision",
            "quantization_config": None,
            "device_map": "auto" if torch.cuda.is_available() else None
        },
        {
            "name": "CPU-only",
            "quantization_config": None,
            "device_map": None,
            "torch_dtype": torch.float32
        }
    ]
    
    for config in configs_to_try:
        try:
            print(f"\nAttempting {config['name']} load...")
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=config.get("quantization_config"),
                trust_remote_code=trust_remote_code,
                device_map=config["device_map"],
                torch_dtype=config.get("torch_dtype", torch_dtype)
            )
            print(f"\n✅ Model loaded successfully ({config['name']})!")
            print_memory()
            return model
        except Exception as e:
            print(f"❌ {config['name']} load failed: {str(e)}")
            continue
    
    raise RuntimeError("All model loading attempts failed")

try:
    model = load_model(MODEL_NAME)
except Exception as e:
    print(f"\n❌ Final model loading error: {str(e)}")
    print("Trying basic CPU load as last resort...")
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="cpu", torch_dtype=torch.float32)
    print("\n✅ Model loaded on CPU")
    print_memory()


=== Loading Model: gpt2 ===
RAM: 10.2% (2.8/31.4GB)

Attempting 4-bit quantized load...
❌ 4-bit quantized load failed: No GPU found. A GPU is needed for quantization.

Attempting 8-bit quantized load...
❌ 8-bit quantized load failed: No GPU found. A GPU is needed for quantization.

Attempting Full precision load...

✅ Model loaded successfully (Full precision)!
RAM: 11.5% (3.1/31.4GB)


In [24]:
# Cell 3: Tokenizer Setup
# =======================

def load_tokenizer(model_name):
    """Load and configure tokenizer"""
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True,
            padding_side="right"
        )
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        print("Tokenizer loaded successfully")
        return tokenizer
    except Exception as e:
        print(f"Tokenizer loading failed: {str(e)}")
        raise

tokenizer = load_tokenizer(MODEL_NAME)

Tokenizer loaded successfully


In [50]:
# Cell 4: Robust Data Preparation from CSV
# =============================================

# 0. Set critical environment variables first
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OMP_NUM_THREADS"] = "1"

# 1. First verify the exact file path
import pandas as pd
from datasets import Dataset

# Check the exact path to your CSV file
dataset_path = "/kaggle/input/database-0522/papers_database.csv"
if not os.path.exists(dataset_path):
    # Try to find the correct path if the default doesn't work
    print("Searching for CSV file...")
    # !find /kaggle/input -name "*.csv" 2>/dev/null
    raise FileNotFoundError(f"Could not find papers_database.csv at {dataset_path}")

# 2. Enhanced dataset preparation with better CSV handling
def prepare_dataset(file_path, max_samples=None):
    """Prepare dataset from CSV with robust error handling"""
    try:
        # First try with pandas for better CSV handling
        df = pd.read_csv(file_path)
        
        # Apply max_samples if specified
        if max_samples is not None and len(df) > max_samples:
            df = df.sample(max_samples)
        
        # Standardize the text column name
        text_col = None
        possible_text_columns = ['text', 'content', 'paper_text', 'abstract', 'body', 'article', 'paper']
        
        for col in possible_text_columns:
            if col in df.columns:
                text_col = col
                break
        
        if text_col is None:
            # Find first string-type column
            for col in df.columns:
                if pd.api.types.is_string_dtype(df[col]):
                    text_col = col
                    break
            else:
                # If no string column found, convert first column to string
                text_col = df.columns[0]
                df[text_col] = df[text_col].astype(str)
        
        # Rename to 'text' if needed
        if text_col != 'text':
            df = df.rename(columns={text_col: 'text'})
        
        # Clean text data
        df['text'] = df['text'].str.strip().replace(r'\s+', ' ', regex=True)
        
        # Convert to HuggingFace Dataset
        dataset = Dataset.from_pandas(df)
        print(f"✅ Loaded dataset with {len(dataset)} samples")
        return dataset
        
    except Exception as e:
        print(f"\n❌ Dataset preparation failed: {str(e)}")
        print("Creating minimal fallback dataset...")
        return Dataset.from_dict({"text": ["Sample text " + str(i) for i in range(10)]})

# 3. Enhanced Tokenization function
def safe_tokenize(examples):
    """Tokenization with multiple fallback strategies"""
    try:
        # First try standard tokenization
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            max_length=512,
            padding="max_length",
            return_tensors="pt"
        )
        
        return {
            "input_ids": tokenized["input_ids"].tolist(),
            "attention_mask": tokenized["attention_mask"].tolist(),
            "labels": tokenized["input_ids"].tolist()
        }
            
    except Exception as e:
        print(f"⚠️ Tokenization warning: {str(e)}")
        # Fallback strategy
        seq_length = min(512, tokenizer.model_max_length)
        return {
            "input_ids": [[tokenizer.pad_token_id or 0] * seq_length],
            "attention_mask": [[1] * seq_length],
            "labels": [[tokenizer.pad_token_id or 0] * seq_length]
        }

# 4. Main processing with comprehensive error handling
try:
    print("\n=== Starting Processing ===")
    
    # Initialize tokenizer if not already defined
    if 'tokenizer' not in globals():
        from transformers import AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Default model
        tokenizer.pad_token = tokenizer.eos_token
    
    # Load and prepare dataset
    dataset = prepare_dataset(dataset_path)
    
    # Small batch test first
    test_size = min(2, len(dataset))
    test_batch = dataset.select(range(test_size))
    test_tokenized = test_batch.map(safe_tokenize, batched=True)
    print("\nTest batch processed successfully!")
    print(f"Sample text: {test_batch[0]['text'][:100]}...")
    print(f"Tokenized length: {len(test_tokenized[0]['input_ids'])}")
    
    # Full dataset processing with progress reporting
    print("\nProcessing full dataset...")
    tokenized_dataset = dataset.map(
        safe_tokenize,
        batched=True,
        batch_size=8,
        remove_columns=['text'],
        writer_batch_size=1000
    )
    
    # Set format for PyTorch
    tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    print("\n✅ Processing completed successfully!")
    print(f"Final dataset features: {tokenized_dataset.features}")
    print(f"Number of samples: {len(tokenized_dataset)}")
    
except Exception as e:
    print(f"\n❌ Critical error in processing: {str(e)}")
    print("Creating minimal fallback dataset...")
    tokenized_dataset = Dataset.from_dict({
        "input_ids": [[0] * 512],
        "attention_mask": [[1] * 512],
        "labels": [[0] * 512]
    })
    tokenized_dataset.set_format(type='torch')
    print("⚠️ Using fallback dataset - model may not train properly")


=== Starting Processing ===
✅ Loaded dataset with 60 samples


Map:   0%|          | 0/2 [00:00<?, ? examples/s]


Test batch processed successfully!
Sample text: a009a494d6d220b47368efac02c33672...
Tokenized length: 512

Processing full dataset...


Map:   0%|          | 0/60 [00:00<?, ? examples/s]


✅ Processing completed successfully!
Final dataset features: {'title': Value(dtype='string', id=None), 'source_url': Value(dtype='string', id=None), 'local_path': Value(dtype='string', id=None), 'text_length': Value(dtype='int64', id=None), 'text_preview': Value(dtype='string', id=None), 'full_text': Value(dtype='string', id=None), 'doc': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
Number of samples: 60


In [51]:
# Cell 5: Training Configuration
# =============================

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# LoRA configuration
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,  
    lora_alpha=32,
    target_modules=["attn.c_attn", "attn.c_proj", "mlp.c_fc", "mlp.c_proj"],  # GPT-2 compatible modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    fan_in_fan_out=True
)

# Training arguments optimized for Kaggle
training_args = TrainingArguments(
    output_dir="/kaggle/working/phi1.5-lora-results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,  # Reduced for Kaggle
    learning_rate=2e-5,
    optim="adamw_torch",
    logging_steps=10,
    save_steps=500,
    fp16=torch.cuda.is_available(),
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    report_to="none"
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Print trainable parameters
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 126,799,104 || trainable%: 1.8606566809809635


In [52]:
# Cell 6: Training Execution
# =========================

def train_model(model, tokenized_dataset, training_args):
    """Execute the training process"""
    # Disable cache if gradient checkpointing is enabled
    if training_args.gradient_checkpointing:
        model.config.use_cache = False
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
    )
    
    print("Starting training...")
    print_memory()
    trainer.train()
    print("Training completed!")
    return trainer

trainer = train_model(model, tokenized_dataset, training_args)

Starting training...
RAM: 12.3% (3.4/31.4GB)


Step,Training Loss
10,8.227700


Training completed!


In [53]:
# Cell 7: Enhanced Model Saving with Shard Support
# ===============================================

def save_model_artifacts(
    model, 
    tokenizer, 
    training_args: Optional[TrainingArguments] = None, 
    output_dir: str = "/kaggle/working/gpt2-lora-trained"
) -> str:
    """
    Save all model artifacts with comprehensive verification.
    Handles both single-file and sharded model formats.
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n💾 Saving model artifacts to: {output_dir}")
    
    # For LoRA models - DON'T merge adapters before saving
    # We want to save the adapter separately
    print("💽 Saving model and adapter...")
    
    # Save the entire model (base model + adapter)
    model.save_pretrained(
        output_dir,
        safe_serialization=True,
        state_dict=model.state_dict()  # Save the complete state including LoRA
    )
    
    # Save tokenizer
    print("🔤 Saving tokenizer...")
    tokenizer.save_pretrained(output_dir)
    
    # Save training arguments if provided
    if training_args is not None:
        print("📝 Saving training arguments...")
        try:
            args_path = os.path.join(output_dir, "training_args.json")
            if hasattr(training_args, 'to_dict'):
                with open(args_path, "w") as f:
                    json.dump(training_args.to_dict(), f, indent=2)
            elif hasattr(training_args, 'to_json_string'):
                with open(args_path, "w") as f:
                    f.write(training_args.to_json_string())
            else:
                print("⚠️ Warning: TrainingArguments has no serialization method")
        except Exception as e:
            print(f"⚠️ Warning: Failed to save training args - {str(e)}")
    
    # Verify the adapter files were saved
    required_files = ['adapter_config.json', 'adapter_model.safetensors']
    missing_files = []
    for file in required_files:
        if not os.path.exists(os.path.join(output_dir, file)):
            missing_files.append(file)
    
    if missing_files:
        print(f"\n⚠️ Warning: Missing adapter files: {missing_files}")
        print("Trying alternative save method...")
        # Explicitly save the adapter
        model.save_pretrained(
            output_dir,
            safe_serialization=True,
            adapter_only=True  # This ensures adapter files are saved
        )
    
    print("\n🔍 Verifying saved files:")
    for file in os.listdir(output_dir):
        size = os.path.getsize(os.path.join(output_dir, file)) / 1024
        print(f"- {file} ({size:.2f} KB)")
    
    return output_dir

In [54]:
# Cell 8: Robust Model Loading and Testing with PEFT support
# ========================================================
def load_and_test_model(
    model_path: str = "/kaggle/working/gpt2-lora-trained", 
    max_length: int = 160,
    test_prompts: Optional[list] = None,
    is_peft_model: bool = True
):
    """
    Load and test a saved model with comprehensive error handling
    """
    print(f"\n🔍 Preparing to load model from: {model_path}")
    
    # Verify model directory exists
    if not os.path.exists(model_path):
        raise ValueError(f"Model directory {model_path} does not exist")
    
    # Show directory contents for debugging
    print("\n📂 Model directory contents:")
    for f in sorted(os.listdir(model_path)):
        size = os.path.getsize(os.path.join(model_path, f)) / 1024
        print(f"- {f} ({size:.2f} KB)")
    
    try:
        print("\n🔄 Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            local_files_only=True
        )
        
        print("\n🔄 Loading model...")
        if is_peft_model:
            # First check if we have adapter files
            adapter_files = [
                f for f in os.listdir(model_path) 
                if f.startswith('adapter_') or f == 'adapter_config.json'
            ]
            
            if not adapter_files:
                print("⚠️ No adapter files found. Loading as regular model.")
                model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    device_map="auto",
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    local_files_only=True
                )
            else:
                print(f"Found adapter files: {adapter_files}")
                # Load base model first
                base_model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    device_map="auto",
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    local_files_only=True
                )
                
                # Then load the PEFT adapter
                model = PeftModel.from_pretrained(
                    base_model,
                    model_path,
                    local_files_only=True
                )
                
                # Merge and unload for inference
                model = model.merge_and_unload()
        else:
            # For regular models
            model = AutoModelForCausalLM.from_pretrained(
                model_path,
                device_map="auto",
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                local_files_only=True
            )
            
        print("\n🎉 Model loaded successfully!")
        
        # Default test prompts if none provided
        if test_prompts is None:
            test_prompts = [
                "What is hardware wallet?? ",
                "What is Proof of Work (PoW)?? ",
                "What is cryptography?? ",
                "What is Peer-to-Peer (P2P)?? ",
                "What is block chain?? ",
                "What is private key?? "
            ]
        
        # Create pipeline - REMOVED device parameter since we're using device_map="auto"
        print("\n🚀 Creating text generation pipeline...")
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
        
        # Run tests
        print("\n🧪 Running generation tests...")
        for i, prompt in enumerate(test_prompts, 1):
            print(f"\n🔹 Test {i}: {prompt}")
            output = pipe(
                prompt,
                max_length=max_length,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                num_return_sequences=1,
                repetition_penalty=1.2
            )
            print("💬 Response:", output[0]['generated_text'])
            
        return model, tokenizer
        
    except Exception as e:
        print(f"\n❌ Critical error loading model: {str(e)}")
        print("\n🛠️ Debugging info:")
        print(f"- Path: {os.path.abspath(model_path)}")
        print(f"- Directory exists: {os.path.exists(model_path)}")
        if os.path.exists(model_path):
            print("- Contents:", os.listdir(model_path))
        raise

In [55]:
# Main execution
if __name__ == "__main__":
    model_path = "/kaggle/working/gpt2-lora-trained"
    
    # Save model artifacts
    save_model_artifacts(model, tokenizer, training_args)
    
    # Load with explicit path and PEFT flag
    load_and_test_model(model_path, is_peft_model=True)
    
    # Test with custom prompts
    custom_prompts = [
        "What is software wallet, and what's the difference between hardware and software wallet? ",
        "What is PoW? ",
        "Explain PoW in 1 sentence. ",
        "Describe the key features of PoW using 3 words. ",
        "What is PoM? Is it something related to cryptography? ",
        "What is a cryptographic product? ",
        "What is P2P? ",
        "What is block chain? ",
        "What is public key, and what's the difference between private and public key? "
    ]
    load_and_test_model(model_path, test_prompts=custom_prompts, is_peft_model=True)


💾 Saving model artifacts to: /kaggle/working/gpt2-lora-trained
💽 Saving model and adapter...
🔤 Saving tokenizer...
📝 Saving training arguments...

🔍 Verifying saved files:
- tokenizer_config.json (0.49 KB)
- adapter_model.safetensors (9234.25 KB)
- adapter_config.json (0.66 KB)
- merges.txt (445.62 KB)
- vocab.json (779.45 KB)
- tokenizer.json (2058.55 KB)
- README.md (4.96 KB)
- training_args.json (3.86 KB)
- special_tokens_map.json (0.13 KB)

🔍 Preparing to load model from: /kaggle/working/gpt2-lora-trained

📂 Model directory contents:
- README.md (4.96 KB)
- adapter_config.json (0.66 KB)
- adapter_model.safetensors (9234.25 KB)
- merges.txt (445.62 KB)
- special_tokens_map.json (0.13 KB)
- tokenizer.json (2058.55 KB)
- tokenizer_config.json (0.49 KB)
- training_args.json (3.86 KB)
- vocab.json (779.45 KB)

🔄 Loading tokenizer...

🔄 Loading model...
Found adapter files: ['adapter_model.safetensors', 'adapter_config.json']

❌ Critical error loading model: We couldn't connect to 'http

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like None is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
notebook_end = time.time()
print(f"Total notebook execution time: {notebook_end - notebook_start:.2f} seconds")